In [1]:
import django
import os
import json
import os.path

In [2]:
os.environ['DJANGO_SETTINGS_MODULE'] = 'django_settings'
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [52]:
import pandas as pd
from kraken.models import *
from django.db.models import F

In [19]:
url = 'https://kraken-storages.s3.ap-northeast-2.amazonaws.com/media/'

In [95]:
datasets = Dataset.objects.filter(sizes__isnull=False, clothes_pictures__isnull=False, wearing_pictures__isnull=False)

In [96]:
datasets.count()

294

In [180]:
size_columns = SizeFeatures.objects.filter(is_active=True).order_by('order').values_list('name',flat=True)
clothes_columns = ClothesFeatures.objects.filter(is_active=True).order_by('order').values_list('name',flat=True)
wearing_columns = WearingFeatures.objects.filter(is_active=True).order_by('order').values_list('name',flat=True)

In [98]:
df_index = []

for q in datasets:
    df_index.append(q.id)

In [185]:
sizes_column = list(size_columns)
clothes_column = list(clothes_columns)
wearing_column = ['w_'+x for x in wearing_columns]

df_column = sizes_column + clothes_column + wearing_column

In [100]:
df_data =[]
for q in datasets:
    size = Size.objects.get(dataset=q)
    clothes = ClothesPicture.objects.get(dataset=q)
    wearing = WearingPicture.objects.get(dataset=q)
    
    sizes = list(size.tags.all().values_list('value',flat=True))
    clothestags = [url+x for x in list(clothes.tags.all().values_list('picture', flat=True))]
    wearingtags = [url+x for x in list(wearing.tags.all().values_list('picture', flat=True))]
    
    data = sizes + clothestags + wearingtags
    df_data.append(data)

In [189]:
df = pd.DataFrame(df_data, index=df_index, columns=df_column)

In [192]:
df.to_pickle('mondeique_virtual_fitting_v0.pickle')

In [193]:
new_df = pd.read_pickle('mondeique_virtual_fitting_v0.pickle')